# Reducing Failure-Inducing Inputs

By construction, fuzzers create inputs that may be hard to read.  This causes issues during _debugging_, when a human has to analyze the exact cause of the failure.  In this chapter, we present techniques that _automatically reduce and simplify failure-inducing inputs to a minimum_ in order to ease debugging.

**Prerequisites**

* The simple "delta debugging" technique for reduction has no specific prerequisites.

This chapter is adapted from [a similar chapter in "The Fuzzing Book"](https://www.fuzzingbook.org/html/Reducer.html). The interface has been simplified to be independent from the `fuzzingbook` infrastructure.

## Synopsis
<!-- Automatically generated. Do not edit. -->

To [use the code provided in this chapter](Importing.ipynb), write

```python
>>> from fuzzingbook.Reducer import <identifier>
```

and then make use of the following features.


A _reducer_ takes a failure-inducing input and reduces it to the minimum that still reproduces the failure.  This chapter provides `Reducer` classes that implement such reducers.

Here is a simple example: An arithmetic expression causes an error in the Python interpreter:

```python
>>> !python -c 'x = 1 + 2 * 3 / 0'
```
Can we reduce this input to a minimum?  To use a `Reducer`, one first has to build a `Runner` whose outcome is `FAIL` if the precise error occurs.  We therefore build a `ZeroDivisionRunner` whose `run()` method will specifically return a `FAIL` outcome if a `ZeroDivisionError` occurs.

```python
>>> from Fuzzer import ProgramRunner
>>> class ZeroDivisionRunner(ProgramRunner):
>>>     """Make outcome 'FAIL' if ZeroDivisionError occurs"""
>>>     def run(self, inp=""):
>>>         result, outcome = super().run(inp)
>>>         if result.stderr.find('ZeroDivisionError') >= 0:
>>>             outcome = 'FAIL'
>>>         return result, outcome
```
If we feed this expression into a `ZeroDivisionRunner`, it will produce an outcome of `FAIL` as designed.

```python
>>> python_input = "x = 1 + 2 * 3 / 0"
>>> python_runner = ZeroDivisionRunner("python")
>>> result, outcome = python_runner.run(python_input)
>>> outcome
```
Delta Debugging is a simple and robust reduction algorithm.  We can tie a `DeltaDebuggingReducer` to this runner, and have it determine the substring that causes the `python` program to fail:

```python
>>> dd = DeltaDebuggingReducer(python_runner)
>>> dd.reduce(python_input)
```
The input is reduced to the minimum: We get the essence of the division by zero.



## Why Reducing?

A common problem in debugging is that given an input, only a _small part of that input may be responsible for the failure_. A central part of debugging is to _identify_ these parts – and to simplify (or _reduce_) the input to a minimal form that reproduces the failure – but does and contains as little else as possible.

Here's an example of such a situation.  We have a `mystery()` method that – given its code – can occasionally fail.  But under which circumstances does this actually happen?  We have deliberately obscured the exact condition in order to make this non-obvious.

In [ ]:
import bookutils

In [ ]:
import re

In [ ]:
def mystery(inp):
    x = inp.find(chr(0o17 + 0o31))
    y = inp.find(chr(0o27 + 0o22))
    if x >= 0 and y >= 0 and x < y:
        raise ValueError("Invalid input")
    else:
        pass

Let us _fuzz_ the function – that is, feed it with random inputs – until we find a failing input. We introduce a simple `fuzz()` function for this purpose:

The function `random.randrange(a, b)` returns a random number in the range (a, b).

In [ ]:
import random

In [ ]:
random.randrange(32, 128)

We can use `random.randrange()` to compose random (printable) characters:

In [ ]:
def fuzz():
    length = random.randrange(10, 70)
    fuzz = ""
    for i in range(length):
        fuzz += chr(random.randrange(32, 127))
    return fuzz

Here are some random strings produced by our `fuzz()` function:

In [ ]:
for i in range(6):
    print(repr(fuzz()))

Let us now use `fuzz()` to find an input where `mistery()` fails:

In [ ]:
while True:
    inp = fuzz()
    try:
        mystery(inp)
    except ValueError:
        break

This is an input that causes `mystery()` to fail:

In [ ]:
failing_input = inp
failing_input

In [ ]:
from ExpectError import ExpectError

In [ ]:
with ExpectError():
    mystery(failing_input)

Something in this input causes `MysteryRunner` to fail.  But what is it?

## Manual Input Reduction

One important step in the debugging process is _reduction_ – that is, to identify those circumstances of a failure that are relevant for the failure to occur, and to _omit_ (if possible) those parts that are not.  As Kernighan and Pike \cite{Kernighan1999} put it:

> For every circumstance of the problem, check whether it is relevant for the problem to occur.  If it is not, remove it from the problem report or the test case in question.

Specifically for inputs, they suggest a _divide and conquer_ process:

> Proceed by binary search.  Throw away half the input and see if the output is still wrong; if not, go back to the previous state and discard the other half of the input.

This is something we can easily try out, using our last generated input:

In [ ]:
failing_input

For instance, we can see whether the error still occurs if we only feed in the first half:

In [ ]:
half_length = len(failing_input) // 2   # // is integer division
first_half = failing_input[:half_length]
first_half

In [ ]:
with ExpectError():
    mystery(first_half)

Nope – the first half alone does not suffice.  Maybe the second half?

In [ ]:
second_half = failing_input[half_length:]
assert first_half + second_half == failing_input
second_half

In [ ]:
with ExpectError():
    mystery(second_half)

This did not go so well either.  We may still proceed by cutting away _smaller chunks_ – say, one character after another.  If our test is deterministic and easily repeated, it is clear that this process eventually will yield a reduced input.  But still, it is a rather inefficient process, especially for long inputs.  What we need is a _strategy_ that effectively minimizes a failure-inducing input – a strategy that can be automated.

## Delta Debugging

One strategy to effectively reduce failure-inducing inputs is _delta debugging_ \cite{Zeller2002}.  Delta Debugging implements the "binary search" strategy, as listed above, but with a twist: If neither half fails (also as above), it keeps on cutting away smaller and smaller chunks from the input, until it eliminates individual characters.  Thus, after cutting away the first half, we cut away
the first quarter, the second quarter, and so on.

Let us illustrate this on our example, and see what happens if we cut away the first quarter.

In [ ]:
quarter_length = len(failing_input) // 4
input_without_first_quarter = failing_input[quarter_length:]
input_without_first_quarter

In [ ]:
with ExpectError():
    mystery(input_without_first_quarter)

Ah! This has failed, and reduced our failing input by 25%.  Let's remove another quarter.

In [ ]:
input_without_first_and_second_quarter = failing_input[quarter_length * 2:]
input_without_first_and_second_quarter

In [ ]:
with ExpectError():
    mystery(input_without_first_and_second_quarter)

This is not too surprising, as we had that one before:

In [ ]:
second_half

In [ ]:
input_without_first_and_second_quarter

How about removing the third quarter, then?

In [ ]:
input_without_first_and_third_quarter = failing_input[quarter_length:
                                                      quarter_length * 2] + failing_input[quarter_length * 3:]
input_without_first_and_third_quarter

In [ ]:
with ExpectError():
    mystery(input_without_first_and_third_quarter)

Yes!  This has succeeded.  Our input is now 50% smaller.

We have now tried to remove pieces that make up $\frac{1}{2}$ and $\frac{1}{4}$ of the original failing string.  In the next iteration, we would go and remove even smaller pieces – $\frac{1}{8}$, $\frac{1}{16}$ and so on.  We continue until we are down to $\frac{1}{26}$ – that is, individual characters.

However, this is something we happily let a computer do for us – and this is what the _Delta Debugging_ algorithm does.  Delta Debugging implements the strategy sketched above: It first removes larger chunks of size $\frac{1}{2}$; if this does not fail, then we proceed to chunks of size $\frac{1}{4}$, then $\frac{1}{8}$ and so on.

Our `ddmin()` implementation uses almost the same Python code as Zeller in \cite{Zeller2002}; the only difference is that it has been adapted to work on Python 3.  The variable `n` (initially 2) indicates the granularity – in each step, chunks of size $\frac{1}{n}$ are cut away.  If none of the test fails (`some_complement_is_failing` is False), then `n` is doubled – until it reaches the length of the input.

In [ ]:
PASS = 'PASS'
FAIL = 'FAIL'
UNRESOLVED = 'UNRESOLVED'

In [ ]:
def ddmin(test, inp, *test_args):
    """Reduce the input inp, using the outcome of test(fun, inp)."""
    assert test(inp, *test_args) != PASS

    n = 2     # Initial granularity
    while len(inp) >= 2:
        start = 0
        subset_length = len(inp) / n
        some_complement_is_failing = False

        while start < len(inp):
            complement = inp[:int(start)] + \
                inp[int(start + subset_length):]

            if test(complement, *test_args) == FAIL:
                inp = complement
                n = max(n - 1, 2)
                some_complement_is_failing = True
                break

            start += subset_length

        if not some_complement_is_failing:
            if n == len(inp):
                break
            n = min(n * 2, len(inp))

    return inp

To see how `reduce()` works, let us run it on our failing input. We need to define a `test` function that returns PASS or FAIL, depending on the test outcome. This `generic_test()` assumes that the function fails if it raises an exception (such as an `AssertException`), and passes otherwise. The optional argument `expected_exc_type` specifies the type of exception to be checked for; this ensures we reduce only for the kind of error raised in the original failure.

In [ ]:
def generic_test(inp, fun, expected_exc_type=None):
    result = None
    try:
        result = fun(inp)
        outcome = PASS
    except Exception as exc:
        if expected_exc_type is None or type(exc) == expected_exc_type:
            outcome = FAIL
        else:
            outcome = UNRESOLVED

    print(f"{fun.__name__}({repr(inp)}): {outcome}")
    return outcome

We can now invoke `ddmin()` in our setting. With each step, we see how the remaining input gets smaller and smaller, until only two characters remain:

In [ ]:
ddmin(generic_test, failing_input, mystery, ValueError)

Now we know why `MysteryRunner` fails – it suffices that the input contains two matching parentheses.  Delta Debugging determines this in 32 steps.  Its result is _1-minimal_, meaning that every character contained is required to produce the error; removing any (as seen in the last two tests, above) no longer makes the test fail.  This property is guaranteed by the delta debugging algorithm, which in its last stage always tries to delete characters one by one.

A reduced test case such as the one above has many advantages:

* A reduced test case __reduces the _cognitive load_ of the programmer__.  The test case is shorter and focused, and thus does not burden the programmer with irrelevant details.  A reduced input typically leads to shorter executions and smaller program states, both of which reduce the search space as it comes to understanding the bug.  In our case, we have eliminated lots of irrelevant input – only the two characters the reduced input contains are relevant.

* A reduced test case __is easier to communicate__.  All one needs here is the summary: `MysteryRunner fails on "()"`, which is much better than `MysteryRunner fails on a 4100-character input (attached)`.

* A reduced test case helps in __identifying duplicates__.  If similar bugs have been reported already, and all of them have been reduced to the same cause (namely that the input contains matching parentheses), then it becomes obvious that all these bugs are different symptoms of the same underlying cause – and would all be resolved at once with one code fix.

How effective is delta debugging?  In the best case (when the left half or the right half fails), the number of tests is logarithmic proportional to the length $n$ of an input (i.e., $O(\log_2 n)$); this is the same complexity as binary search.  In the worst case, though, delta debugging can require a number of tests proportional to $n^2$  (i.e., $O(n^2)$) – this happens in the case when we are down to character granularity, and we have to repeatedly tried to delete all characters, only to find that deleting the last character results in a failure \cite{Zeller2002}.  (This is a pretty pathological situation, though.)

In general, delta debugging is a robust algorithm that is easy to implement, easy to deploy, and easy to use – provided that the underlying test case is deterministic and runs quickly enough to warrant a number of experiments. In general, any debugging task should start with simplifying the test case as much as possible – and this is where delta debugging can help.

## A Simple Interface

As defined above, using `ddmin()` still requires the developer to set up a special testing function. We want to simplify the setup such that only a single Python line is required.

Our aim is to have a `DeltaDebugger` class that we can use in conjunction with a failing (i.e., exception raising) function call:

```python
with DeltaDebugger():
    mystery(failing_input)
```
Here, at the end of the `with` statement, the debugger would print out the minimal input that causes the failure.

### Collecting a Call

In [ ]:
import sys

In [ ]:
from types import FunctionType

In [ ]:
class CallCollector(object):
    def __init__(self):
        """Reduce a function call."""
        self._function = None
        self._args = None
        self._exception = None

    def traceit(self, frame, event, arg):
        """Tracing function. Collect first call."""
        if event == 'call':
            name = frame.f_code.co_name
            if name.startswith('__'):
                # Internal function
                return

            self._function = FunctionType(frame.f_code,
                                          globals=globals(),
                                          name=name)
            self._args = frame.f_locals

            # Turn tracing off
            sys.settrace(self.original_trace_function)

    def diagnosis(self):
        """Produce a diagnosis. To be defined in subclasses."""
        pass

    def __enter__(self):
        """Called at begin of `with` block. Turn tracing on."""
        self.original_trace_function = sys.gettrace()
        sys.settrace(self.traceit)

    def __exit__(self, exc_type, exc_value, traceback):
        """Called at end of `with` block. Turn tracing off."""
        sys.settrace(self.original_trace_function)
        if exc_type is not None and self._function is None:
            raise exc_value

        self._exception = exc_value
        self.diagnosis()
        return True  # Ignore exception

In [ ]:
call_collector = CallCollector()
with call_collector:
    mystery(failing_input)

In [ ]:
call_collector._function

In [ ]:
call_collector._args

In [ ]:
call_collector._exception

How robust is this?

In [ ]:
with ExpectError():
    c = CallCollector()
    with c:
        some_error()

### Repeating a Call

In [ ]:
with ExpectError():
    call_collector._function("foo")

In [ ]:
with ExpectError():
    call_collector._function(failing_input)

In [ ]:
with ExpectError():
    call_collector._function(**call_collector._args)

In [ ]:
class CallCollector(CallCollector):
    def call(self, new_args={}):
        args = {}
        for var in self._args:
            args[var] = self._args[var]
        for var in new_args:
            args[var] = new_args[var]

        return self._function(**new_args)

In [ ]:
call_collector = CallCollector()
with call_collector:
    mystery(failing_input)

In [ ]:
with ExpectError():
    call_collector.call({'inp': 'foo'})

### Reducing Inputs

We first introduce a `Reducer` class as an abstract superclass for all kinds of reducers.

The `test()` method runs a single test (with logging, if wanted); the `reduce()` method will eventually reduce an input to the minimum.

In [ ]:
class Reducer(CallCollector):
    def __init__(self, log=False):
        super().__init__()
        self.log = log
        self.reset()

    def reset(self):
        self.tests = 0

    def run(self, args):
        try:
            self.call(args)
        except Exception as exc:
            if type(exc) == type(self._exception):
                return FAIL
            else:
                return UNRESOLVED  # Some other failure

        return PASS

    def format_call(self, args=None):
        if args is None:
            args = self._args
        return self._function.__name__ + "(" + \
            ", ".join(f"{arg}={repr(args[arg])}" for arg in args) + ")"

    def test(self, args):
        outcome = self.run(args)
        self.tests += 1
        if self.log:
            print(f"Test #{self.tests} {self.format_call(args)}: {outcome}")

        return outcome

The `CachingReducer` variant saves test results, such that we don't have to run the same tests again and again:

In [ ]:
class CachingReducer(Reducer):
    def reset(self):
        super().reset()
        self.cache = {}

    def test(self, args):
        index = ((k, v) for k, v in args.items())
        if index in self.cache:
            return self.cache[index]

        outcome = super().test(args)
        self.cache[index] = outcome
        return outcome

Here comes the _Delta Debugging_ reducer.  Delta Debugging implements the strategy sketched above: It first removes larger chunks of size $\frac{1}{2}$; if this does not fail, then we proceed to chunks of size $\frac{1}{4}$, then $\frac{1}{8}$ and so on.

In [ ]:
class DeltaDebugger(CachingReducer):
    def reduce(self, var_to_be_reduced, args):
        self.reset()
        assert self.test(args) != PASS, f"{self.format_call(args)} cannot pass"
        inp = args[var_to_be_reduced]

        n = 2     # Initial granularity
        while len(inp) >= 2:
            start = 0
            subset_length = len(inp) / n
            some_complement_is_failing = False

            while start < len(inp):
                complement = inp[:int(start)] + \
                    inp[int(start + subset_length):]

                new_args = {}
                for var in args:
                    new_args[var] = args[var]
                new_args[var_to_be_reduced] = complement
                if self.test(new_args) == FAIL:
                    inp = complement
                    n = max(n - 1, 2)
                    some_complement_is_failing = True
                    break

                start += subset_length

            if not some_complement_is_failing:
                if n == len(inp):
                    break
                n = min(n * 2, len(inp))

        return inp

In [ ]:
class DeltaDebugger(DeltaDebugger):
    def reducible(self, arg):
        try:
            x = len(arg)
        except TypeError:
            return False
        
        try:
            x = arg[0]
        except TypeError:
            return False
        except IndexError:
            return False
        
        return True

In [ ]:
class DeltaDebugger(DeltaDebugger):
    def reduced_args(self):
        args = self._args
        vars_to_be_reduced = set(args.keys())
        
        while len(vars_to_be_reduced) > 0:
            for var in vars_to_be_reduced:
                value = args[var]
                if not self.reducible(value):
                    vars_to_be_reduced.remove(var)
                    break
                if self.log:
                    print(f"Reducing {var}...")
                reduced_value = self.reduce(var, args)
                if len(reduced_value) < len(value):
                    args[var] = reduced_value
                    vars_to_be_reduced = set(args.keys())
                vars_to_be_reduced.remove(var)
                break

        assert self.test(args) == FAIL, f"{self.format_call(args)} does not fail"
        return args

In [ ]:
class DeltaDebugger(DeltaDebugger):
    def diagnosis(self):
        assert self._function is not None, "No function call observed"
        assert self._exception is not None, f"{self.format_call()} did not raise an exception"

        reduced_args = self.reduced_args()
        print(self.format_call(reduced_args))
        return reduced_args

To see how the `DeltaDebuggingReducer` works, let us run it on our failing input.  With each step, we see how the remaining input gets smaller and smaller, until only two characters remain:

In [ ]:
with DeltaDebugger(log=True):
    mystery('()')

In [ ]:
with DeltaDebugger(log=True):
    mystery(failing_input)

In [ ]:
def string_error(s1, s2):
    assert s1 not in s2

In [ ]:
with DeltaDebugger(log=True):
    string_error("foo", "foobar")

In [ ]:
def list_error(l1, l2, maxlen):
    assert len(l1) < len(l2) < maxlen

In [ ]:
with DeltaDebugger(log=True):
    list_error(l1=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], l2=[1, 2, 3], maxlen=5)

## Synopsis

A _reducer_ takes a failure-inducing input and reduces it to the minimum that still reproduces the failure.  This chapter provides a `DeltaDebugger` class that implements such a reducer.

Here is a simple example: An arithmetic expression causes an error in the Python interpreter:

In [ ]:
def myeval(inp):
    return eval(inp)

In [ ]:
with ExpectError():
    myeval('1 + 2 * 3 / 0')

Can we reduce this input to a minimum? Delta Debugging is a simple and robust reduction algorithm. We can apply a `DeltaDebugger` to this call, and have it determine the minimal argument that still causes the function to fail:

In [ ]:
with DeltaDebugger():
    myeval('1 + 2 * 3 / 0')

The input is reduced to the maximum: We get the essence of the division by zero.

## Lessons Learned

* Reducing failure-inducing inputs to a minimum is helpful for testing and debugging.
* _Delta debugging_ is a simple and robust algorithm to easily reduce test cases.
* For syntactically complex inputs, _grammar-based reduction_ is much faster and yields better results.

## Next Steps

Our next chapter focuses on [Web GUI Fuzzing](WebFuzzer.ipynb), another domain where generating and reducing test cases is central.

## Background

The "lexical" delta debugging algorithm discussed here stems from \cite{Zeller2002}; actually, this is the exact Python implementation as used by Zeller in 2002.  The idea of systematically reducing inputs has been discovered a number of times, although not as  automatic and generic as delta debugging. \cite{Slutz1998}, for instance, discusses systematic reduction of SQL statements for SQL databases; the general process as manual work is well described by \cite{Kernighan1999}.

The deficits of delta debugging as it comes to syntactically complex inputs were first discussed in *compiler testing*, and _reducing tree inputs_ rather than string inputs was quickly discovered as an alternative.  *Hierarchical Delta Debugging* (*HDD*) \cite{Misherghi2006} applies delta debugging on subtrees of a parse tree, systematically reducing a parse tree to a minimum.  _Generalized Tree Reduction_ \cite{Herfert2017} generalizes this idea to apply arbitrary _patterns_ such as replacing a term by a compatible term in a subtree, as `subtrees_with_symbol()` does.  Using _grammars_ to reduce inputs was first implemented in the _Perses_ tool \cite{Sun2018}; our algorithm implements very similar strategies.  Searching for alternate expansions (as `alternate_reductions()`) is a contribution of the present chapter.

While `GrammarReducer` is a generic approach that can be parameterized with an arbitrary grammar, _language-specific_ approaches can do a much better job for the language at hand.  *C-Reduce* \cite{Regehr2012} is a reducer specifically targeting the reduction of programming languages.  Besides reductions in the style of delta debugging or tree transformations, C-Reduce comes with more than 30 source-to-source transformations that replace aggregates by scalars, remove function parameters at a definition and all call sites, change functions to return `void` and deleting all `return` statements, and many more.  While specifically instantiated for the C language (and used for testing C compilers), these principles extend to arbitrary programming languages following an ALGOL-like syntax.  When testing a compiler, C-Reduce is the tool to go for.

This [blog post](https://www.drmaciver.com/2019/01/notes-on-test-case-reduction/) by David McIver contains lots of insights on how to apply reduction in practice, in particular multiple runs with different abstraction levels.

## Exercises

How to best reduce inputs is still an underdeveloped field of research, with lots of opportunities.

### Exercise 1: Mutation-Based Fuzzing with Reduction

When fuzzing with a population, it can be useful to occasionally _reduce_ the length of each element, such that future descendants are shorter, too, which typically speeds up their testing.

Consider the `MutationFuzzer` class from [the chapter on mutation-based fuzzing](MutationFuzzer.ipynb). 
Extend it such that whenever a new input is added to the population, it is first reduced using delta debugging.

**Solution.** Left to the reader.

### Exercise 2: Reduction by Production

Grammar-based input reduction, as sketched above, might be a good algorithm, but is by no means the only alternative.  One interesting question is whether "reduction" should only be limited to elements already present, or whether one would be allowed to also create _new_ elements.  These would not be present in the original input, yet still allow to produce a much smaller input that would still reproduce the original failure.

As an example, consider the following grammar:

```
<number> ::= <float> | <integer> | <not-a-number>
<float> ::= <digits>.<digits>
<integer> ::= <digits>
<not-a-number> ::= NaN
<digits> ::= [0-9]+
```

Assume the input `100.99` fails.  We might be able to reduce it to a minimum of, say, `1.9`.  However, we cannot reduce it to an `<integer>` or to `<not-a-number>`, as these symbols do not occur in the original input.  By allowing to _create_ alternatives for these symbols, we could also tests inputs such as `1` or `NaN` and further generalize the class of inputs for which the program fails.

Create a class `GenerativeGrammarReducer` as subclass of `GrammarReducer`; extend the method `reduce_subtree()` accordingly.

**Solution.** Left to the reader.

### Exercise 3: The Big Reduction Shoot-Out

Create a _benchmark_ for the grammars already defined earlier, consisting of:

1. A set of _inputs_, produced from these very grammars using `GrammarFuzzer` and derivatives;
2. A set of _tests_ which check for the occurrence of individual symbols as well as pairs and triples of these symbols:
    * Tests should be _unresolved_ if the input is not syntactically valid;
    * Tests should _fail_ if the symbols (or pairs or triples thereof) occur;
    * Tests should _pass_ in all other cases.
    
Compare delta debugging and grammar-based debugging on the benchmark.  Implement HDD \cite{Misherghi2006} and _Generalized Tree Reduction_ \cite{Herfert2017} and add them to your comparison.  Which approach performs best, and under which circumstances?

**Solution.** Left to the reader.